In [ ]:
import glob
import os

import matplotlib.pyplot as plt
from multiprocessing import Pool
import numpy as np
from scipy.interpolate import RectBivariateSpline
from tqdm import tqdm

%matplotlib inline

In [ ]:
sid_dir = 'OSISAF_ice_drift_CDR_postproc'
lag_dir = 'NERSC_arctic25km_sea_ice_age_v2p1'

mesh_files = sorted(glob.glob(f'{lag_dir}/mesh/2025/mesh_*npz'))
print(len(mesh_files), mesh_files[0], mesh_files[-1])

# Load all inital values
mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc']
landmask = np.load(mesh_init_file)['landmask']
min_lm = 0.5
force = False

In [ ]:
def interpolate(mesh_file):
    basename = os.path.basename(mesh_file)
    sic_dst_file = mesh_file.replace("mesh_", "sic_").replace("/mesh/", "/sic/")
    if os.path.exists(sic_dst_file) and not force:
        return
    print(sic_dst_file)
    mesh_date = basename.split('.')[0].split('_')[-1]
    mesh_year = mesh_date[:4]
    # load mesh
    with np.load(mesh_file) as data:
        x = data['x']
        y = data['y']
        t = data['t']

    # load SIC
    file_mask = f'{sid_dir}/{mesh_year}/ice_drift_nh_ease*{mesh_date}1200.nc.npz'
    try:
        sic_src_file = glob.glob(file_mask)[0]
    except:
        raise ValueError(f'Cannot find {file_mask}')
    try:
        with np.load(sic_src_file) as data:
            cgrd = data['c']
    except:
        print(f'Cannot load c from {sic_src_file}')
        raise ValueError
    cgrd[np.isnan(cgrd)] = 0

    # interpolate SIC
    try:
        c = RectBivariateSpline(xc, yc, cgrd[::-1], kx=1, ky=1)(y[t].mean(axis=1), x[t].mean(axis=1), grid=False)
    except:
        raise ValueError(f'Fail to interpolate SIC in {mesh_file}')
    # interpolate landmask
    try:
        lm = RectBivariateSpline(xc, yc, landmask[::-1], kx=1, ky=1)(y[t].mean(axis=1), x[t].mean(axis=1), grid=False)
    except:
        raise ValueError(f'Fail to interpolate LM in {mesh_file}')
    # clear land elements
    c[lm > min_lm] = 0
    os.makedirs(os.path.split(sic_dst_file)[0], exist_ok=True)
    np.savez(sic_dst_file, c=c)

for mesh_file in tqdm(mesh_files, desc="Interpolating SIC"):
    interpolate(mesh_file)

In [ ]:
year = '2025'
date = '0731'
cgrd = np.load(glob.glob(f'{sid_dir}/{year}/ice_drift_nh_ease*{year}{date}1200.nc.npz')[0])['c']
c = np.load(f'{lag_dir}/sic/{year}/sic_{year}{date}.npz')['c']
x = np.load(f'{lag_dir}/mesh/{year}/mesh_{year}{date}.npz')['x']
y = np.load(f'{lag_dir}/mesh/{year}/mesh_{year}{date}.npz')['y']
t = np.load(f'{lag_dir}/mesh/{year}/mesh_{year}{date}.npz')['t']

fig, ax = plt.subplots(1,2, figsize=(20,10))
ax[0].imshow(cgrd, extent=[xc.min(), xc.max(), yc.min(), yc.max()])
ax[1].tripcolor(x, y, c, triangles=t)

for a in ax:
    a.set_xlim([-1500, 1500])
    a.set_ylim([-1500, 1500])
plt.show()